# Compute Persistence Homology Barcode

## Step 1: Upload Structure

In [1]:
from aiida_widgets import StructureUploadWidget
import ipywidgets as ipw
from IPython.display import display


upload_widget = StructureUploadWidget(node_class='CifData')
display(upload_widget)

StructureUploadWidget(children=(FileUploadWidget(label=u'Upload Structure', _dom_classes=('widget_item', 'btn-group')), Output(), NGLWidget(), HBox(children=(Button(description=u'Store in AiiDA', style=ButtonStyle()), Text(value=u'', placeholder=u'Label (optional)')))))

## Step 2: Submit Calculation

In [2]:
from aiida_phtools.workflows.dmatrix import DistanceMatrixWorkChain



#import aiida_phtools.tests as pt
#import aiida_zeopp.tests as zt
#import aiida_gudhi.tests as gt
from IPython.display import display
from aiida_widgets import CodeDropdown

zeopp_dd = CodeDropdown(input_plugin='zeopp.network')
pore_surface_dd = CodeDropdown(input_plugin='phtools.surface')
distance_matrix_dd = CodeDropdown(input_plugin='phtools.dmatrix')
rips_code_dd = CodeDropdown(input_plugin='gudhi.rdm')

outputs=None
def on_click_submit(change):
    global outputs
    from aiida.work.run import run, submit

    with submit_output:
        print("Running Workchain for structure {}".format(str(upload_widget.structure_node)))
        print("Please stand by ... (this can take a few minutes)")
        outputs = run(
            DistanceMatrixWorkChain,
            structure=upload_widget.structure_node,
            zeopp_code=zeopp_dd.selected_code,
            pore_surface_code=pore_surface_dd.selected_code,
            distance_matrix_code=distance_matrix_dd.selected_code,
            rips_code=rips_code_dd.selected_code,
        )
        print("Workchain completed!")
    
btn_submit = ipw.Button(description='Compute barcodes')
btn_submit.on_click(on_click_submit)
submit_output = ipw.Output()
#text_calc_name = ipw.Text(description='Calculation label: ', placeholder='A great name.',
#                          style=style, layout=layout)


display(zeopp_dd, pore_surface_dd, distance_matrix_dd, rips_code_dd, btn_submit, submit_output)

CodeDropdown(children=(HBox(children=(Label(value=u'Select code:'), Dropdown(options=('network@localhost', 'zeopp@localhost', 'zeopp-0.3@quantum-mobile'), value='network@localhost'), Output())),))

CodeDropdown(children=(HBox(children=(Label(value=u'Select code:'), Dropdown(options=('pore_surface@localhost', 'pore_surface_executable@localhost'), value='pore_surface@localhost'), Output())),))

CodeDropdown(children=(HBox(children=(Label(value=u'Select code:'), Dropdown(options=('distance_matrix@localhost', 'distance_matrix_executable@localhost'), value='distance_matrix@localhost'), Output())),))

CodeDropdown(children=(HBox(children=(Label(value=u'Select code:'), Dropdown(options=('rips_distance_matrix_persistence@localhost',), value='rips_distance_matrix_persistence@localhost'), Output())),))

Button(description=u'Compute barcodes', style=ButtonStyle())

Output()

## Step 3: Show Barcodes

In [ ]:
%matplotlib inline

#from aiida.orm import load_node
#outputs = { 'barcode': load_node(4713)}

btn_plot = ipw.Button(description='Show barcodes')

def on_click_plot(change):
    with plot_output:
        
        if outputs is None:
            print("No barcode found. Has the calculation finished?")
            return
        

        from aiida_gudhi.parsers.barcode import BarcodeParser
        parser = BarcodeParser(filename=outputs['barcode'].get_file_abs_path(),max_life=4.2)
    
        for d in [0,1,2]:
            parser.plot(d)
        
btn_plot.on_click(on_click_plot)
plot_output = ipw.Output()
display(btn_plot, plot_output)